In [7]:
import openslide
import numpy as np
import os
import cv2
import json

In [132]:
# 定义svs转jpg函数
def svs_mask2jpg(folderpath, filename, json_path, mode): 
    wsi_filepath = os.path.join(folderpath, filename)
    wsi_handle = openslide.OpenSlide(wsi_filepath)
    if len(wsi_handle.level_downsamples)>3:
        level = 3
    else:
        level = len(wsi_handle.level_downsamples)-1
    W, H = wsi_handle.level_dimensions[level]
        
    if mode == 'test':
        annotation_dict = read_json(json_path, mode)

        keys = list(annotation_dict.keys())
        imgs = {}
        for i in range(len(keys)):
            #print(annotation_dict[keys[i]])
            region_points = [(annotation_dict[keys[i]]/4**level).astype(np.int32)]
            mask = np.zeros((H, W))
            cv2.fillPoly(mask, region_points, (255,255,255))
            npy_slide = np.array(
                wsi_handle.read_region(
                    location=(0, 0), 
                    level=level, 
                    size=wsi_handle.level_dimensions[level])
                )[:, :, :3]
            # cast to unint8
            img = npy_slide.astype(np.uint8)
            img = cv2.add(img, np.zeros(np.shape(img), dtype=np.uint8), mask=mask.astype(np.uint8))
            # 如果图片过大，则resize一下
            while img.shape[0]>2000:
                img=cv2.resize(img,(int(img.shape[0]/2),int(img.shape[1]/2)))
            imgs[keys[i]] = img
    else:
        annotation_list = read_json(json_path, mode)
    
        imgs = []
        for i in range(len(annotation_list)):
        
            region_points = [(annotation_list[i]/4**level).astype(np.int32)]
            mask = np.zeros((H, W))
            cv2.fillPoly(mask, region_points, (255,255,255))
            npy_slide = np.array(
                wsi_handle.read_region(
                    location=(0, 0), 
                    level=level, 
                    size=wsi_handle.level_dimensions[level])
                )[:, :, :3]
            # cast to unint8
            img = npy_slide.astype(np.uint8)
            img = cv2.add(img, np.zeros(np.shape(img), dtype=np.uint8), mask=mask.astype(np.uint8))
            # 如果图片过大，则resize一下
            while img.shape[0]>2000:
                img=cv2.resize(img,(int(img.shape[0]/2),int(img.shape[1]/2)))
            imgs.append(img)
            
    return imgs

def read_json(json_path, mode):
    f = open(json_path, 'r', encoding='utf-8')
    annotation = json.load(f)
    annotation_list = []
    annotation_dict = {}
    for idx, item in enumerate(annotation['contexts']):
        tem_list = []
        for coordinate in item['points']:
            tem_list.append([coordinate['x'], coordinate['y']])
        region = np.array(tem_list)

        if mode == 'test':
            annotation_dict[item['name']] = region
        else:
            annotation_list.append(region)
            
    if mode == 'test':
        return annotation_dict
    else:
        return annotation_list

# 定义svs转jpg函数
def svs2jpg(folderpath, filename): 
    wsi_filepath = os.path.join(folderpath, filename)
    wsi_handle = openslide.OpenSlide(wsi_filepath)
    print(len(wsi_handle.level_downsamples))
    if len(wsi_handle.level_downsamples)>3:
        level = 3
    else:
        level = len(wsi_handle.level_downsamples)-1
    npy_slide = np.array(
        wsi_handle.read_region(
            location=(0, 0), 
            level=level, 
            size=wsi_handle.level_dimensions[level])
        )[:, :, :3]
    # cast to unint8
    img = npy_slide.astype(np.uint8)
    # 如果图片过大，则resize一下
    while img.shape[0]>2000:
        img=cv2.resize(img,(int(img.shape[0]/2),int(img.shape[1]/2)))
    return img

In [135]:
# 查看某一svs文件属性
folderpath = "E:/3rdSEED/test/test/kfb/svs"  # svs图像路径
filename = "B5MAwD.svs"
json_path = "E:/3rdSEED/test/test/json/B5MAwD.json"

mode = 'test'
# 调用svs2jpg函数
imgs = svs_mask2jpg(folderpath, filename, json_path, mode)

if mode == 'test':
    keys = list(imgs.keys())
    print(keys)
    for i in range(len(keys)):
        print(imgs[keys[i]].shape)
        print(keys[i].replace(" ", ""))
        cv2.imshow('img',imgs[keys[i]])
        cv2.waitKey(0)
else:
    for i in range(len(imgs)):
        print(imgs[i].shape)
        cv2.imshow('img',imgs[i])
        cv2.waitKey(0)

['Annotation 0', 'Annotation 1', 'Annotation 2', 'Annotation 3']
(1157, 1498, 3)
Annotation0
(1157, 1498, 3)
Annotation1
(1157, 1498, 3)
Annotation2
(1157, 1498, 3)
Annotation3


In [141]:
folderpath = "E:/3rdSEED/t1/kfb/svs"  # svs图像路径 注意路径不要有中文字符 斜杠要为 /
savepath = "D:/AI/3rdSEED/data_svs_mask2jpg/T1"  # jpg图像保存路径
json_folder = "E:/3rdSEED/t1/含T1标注的json/"

path = [["E:/3rdSEED/T0/svs","D:/AI/3rdSEED/data_svs_mask2jpg/T0","",'train'],
        ["E:/3rdSEED/t1/kfb/svs","D:/AI/3rdSEED/data_svs_mask2jpg/T1","E:/3rdSEED/t1/含T1标注的json/",'train'],
        ["E:/3rdSEED/t2/kfb/svs","D:/AI/3rdSEED/data_svs_mask2jpg/T2","E:/3rdSEED/t2/含T2标注的json/",'train'],
        ["E:/3rdSEED/t3/kfb/svs","D:/AI/3rdSEED/data_svs_mask2jpg/T3","E:/3rdSEED/t3/含T3标注的json/",'train'],
        ["E:/3rdSEED/tis/kfb/svs","D:/AI/3rdSEED/data_svs_mask2jpg/Tis","E:/3rdSEED/tis/含Tis标注的json/",'train'],
        ["E:/3rdSEED/test/test/kfb/svs","D:/AI/3rdSEED/data_svs_mask2jpg/Test","E:/3rdSEED/test/test/json/",'test']
        ]

for i in range(len(path)):
    folderpath, savepath, json_folder, mode = path[i][0], path[i][1], path[i][2], path[i][3]

    # 如果保存路径不存在则创建文件夹
    if not os.path.exists(savepath):
        os.makedirs(savepath)

    # 遍历文件夹中的文件并做转换
    for root, dirs, files in os.walk(folderpath):
        for f in files:        
            print(os.path.join(root, f))
            file_name = f.split('.')[0]
            json_path = json_folder + file_name + '.json'
            if os.path.exists(json_path):
                imgs = svs_mask2jpg(root, f, json_path, mode)
                if mode == 'test':
                    keys = list(imgs.keys())
                    for i in range(len(keys)):
                        cv2.imwrite(os.path.join(savepath, file_name + '_' + keys[i].replace(" ", "") + '.jpg'), imgs[keys[i]])
                else:
                    img = svs2jpg(root, f)
                    cv2.imwrite(os.path.join(savepath, file_name + '.jpg'), img)
                    for i in range(len(imgs)):
                        cv2.imwrite(os.path.join(savepath, file_name + '_' + str(i) + '.jpg'), imgs[i])
            else:
                img = svs2jpg(root, f)
                cv2.imwrite(os.path.join(savepath, file_name + '.jpg'), img)

E:/3rdSEED/T0/svs\0bJOE7.svs
7
E:/3rdSEED/T0/svs\0EDZFS.svs
7
E:/3rdSEED/T0/svs\0eFPg4.svs
7
E:/3rdSEED/T0/svs\0pwioC.svs
7
E:/3rdSEED/T0/svs\0ynI8A.svs
7
E:/3rdSEED/T0/svs\1gwBov.svs
7
E:/3rdSEED/T0/svs\1wfjrZ.svs
7
E:/3rdSEED/T0/svs\1yRj90.svs
7
E:/3rdSEED/T0/svs\24YhOR.svs
7
E:/3rdSEED/T0/svs\2afRKJ.svs
7
E:/3rdSEED/T0/svs\2dp7Ho.svs
7
E:/3rdSEED/T0/svs\2dQDn1.svs
7
E:/3rdSEED/T0/svs\2Q3vbU.svs
7
E:/3rdSEED/T0/svs\2R7Adz.svs
6
E:/3rdSEED/T0/svs\2WFRmu.svs
7
E:/3rdSEED/T0/svs\3N2dJL.svs
2
E:/3rdSEED/T0/svs\3OVsha.svs
7
E:/3rdSEED/T0/svs\3T0faH.svs
7
E:/3rdSEED/T0/svs\3tuv07.svs
7
E:/3rdSEED/T0/svs\3w9WHG.svs
6
E:/3rdSEED/T0/svs\3XbmRu.svs
7
E:/3rdSEED/T0/svs\4YkgEn.svs
7
E:/3rdSEED/T0/svs\4yOXZj.svs
7
E:/3rdSEED/T0/svs\50sx2S.svs
7
E:/3rdSEED/T0/svs\52C1af.svs
7
E:/3rdSEED/T0/svs\6kaxlQ.svs
7
E:/3rdSEED/T0/svs\6kux3B.svs
7
E:/3rdSEED/T0/svs\6oGP1C.svs
6
E:/3rdSEED/T0/svs\6q0Imz.svs
7
E:/3rdSEED/T0/svs\7d6rna.svs
7
E:/3rdSEED/T0/svs\7GzdeM.svs
7
E:/3rdSEED/T0/svs\7WSmJO.svs
6
E:/3rdSE